In [1]:
%load_ext autoreload
%autoreload 2

In [78]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
import tensorflow as tf
from policy import ConvNet
from concepts import DynamicConcepts, linear_regression
import absl.logging
import json
import string
import random
import numpy as np
import os
from env import gogame
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

session_name = "falcon"
model_name = "net"
board_size = 5
board_name = f'{board_size}x{board_size}'
agents_to_sample = [10, 20, 60, 100, 500, 1000]

# Mcts simulations
simulations = 2000

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

mcts_model_path = f"../models/best_models/board_size_{board_size}/net_1000.keras"

init_state, concept_type_single = DynamicConcepts.ko_fight(board_size)

print("Initial state:\n", gogame.str(init_state))

dynamic_concept = DynamicConcepts(init_state, simulations, board_size, concept_type_single, mcts_model_path)

CONCEPT_NAME = DynamicConcepts.ko_fight.__name__

Initial state:
 	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─●─○─┼─╢
2	●─┼─●─○─○
3	╟─●─○─┼─╢
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 4, White Area: 5

INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpleq2xgvg/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpleq2xgvg/assets
2024-02-11 15:56:08.854116: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpleq2xgvg
2024-02-11 15:56:08.856150: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-11 15:56:08.856161: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpleq2xgvg
2024-02-11 15:56:08.863475: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-11 15:56:08.909251: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpleq2xgvg
2024-02-11 15:56:08.926267: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 72151 microseconds.


In [79]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [80]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (11, 5, 5, 5)
Negative cases:  (8, 5, 5, 5)


In [81]:
# If larger than 50, do not visualize the tree do to the size (crashes the kernel)
if simulations <= 100:
    graph = dynamic_concept.view_tree()

    graph.render('./visualization/images/tree', view=True)

In [36]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [37]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [ ]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print(gogame.str(negative_cases[i], nn_format=True))
    print()

In [85]:
BINARY = True

# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=True,
    epochs=100,
    dynamic=True
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/dynamic", exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Positions to consider:  19
Epoch 1/100


2024-02-11 15:58:13.720695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:58:13.752981: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 672ms/step - loss: 0.9129
Epoch 2/100
1/1 [==============================] - 0s 57ms/step - loss: 0.9124
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9118
Epoch 4/100
1/1 [==============================] - 0s 61ms/step - loss: 0.9112
Epoch 5/100
1/1 [==============================] - 0s 63ms/step - loss: 0.9106
Epoch 6/100
1/1 [==============================] - 0s 59ms/step - loss: 0.9101
Epoch 7/100
1/1 [==============================] - 0s 24ms/step - loss: 0.9095
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 0.9089
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 0.9084
Epoch 10/100
1/1 [==============================] - 0s 23ms/step - loss: 0.9078
Epoch 11/100
1/1 [==============================] - 0s 22ms/step - loss: 0.9072
Epoch 12/100
1/1 [==============================] - 0s 22ms/step - loss: 0.9067
Epoch 13/100
1/1 [==============================] - 0s 43ms/

2024-02-11 15:58:16.798885: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [86]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=100,
            dynamic=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/dynamic", exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)

Getting activation outputs: 100%|██████████| 1/1 [00:00<00:00, 60.85it/s]

Performing regression for layer 0
Epoch 1/100



2024-02-11 15:58:20.730953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:58:20.784524: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 831ms/step - loss: 1.1332
Epoch 2/100
1/1 [==============================] - 0s 52ms/step - loss: 1.1316
Epoch 3/100
1/1 [==============================] - 0s 55ms/step - loss: 1.1301
Epoch 4/100
1/1 [==============================] - 0s 56ms/step - loss: 1.1285
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1270
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1255
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 1.1240
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1225
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1210
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1195
Epoch 11/100
1/1 [==============================] - 0s 61ms/step - loss: 1.1180
Epoch 12/100
1/1 [==============================] - 0s 41ms/step - loss: 1.1165
Epoch 13/100
1/1 [==============================] - 0s 29ms/

2024-02-11 15:58:23.985574: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:58:24.330356: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:58:24.363503: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

2024-02-11 15:59:19.338252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:59:19.687433: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:59:19.720694: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 750ms/step - loss: 1.1458
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1428
Epoch 3/100
1/1 [==============================] - 0s 56ms/step - loss: 1.1399
Epoch 4/100
1/1 [==============================] - 0s 47ms/step - loss: 1.1370
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1340
Epoch 6/100
1/1 [==============================] - 0s 49ms/step - loss: 1.1311
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1282
Epoch 8/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1253
Epoch 9/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1225
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1196
Epoch 11/100
1/1 [==============================] - 0s 57ms/step - loss: 1.1168
Epoch 12/100
1/1 [==============================] - 0s 53ms/step - loss: 1.1139
Epoch 13/100
1/1 [==============================] - 0s 27ms/

2024-02-11 15:59:23.000328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:59:23.363662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-11 15:59:23.396527: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 732ms/step - loss: 1.1575
Epoch 2/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1548
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1521
Epoch 4/100
1/1 [==============================] - 0s 52ms/step - loss: 1.1494
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1467
Epoch 6/100
1/1 [==============================] - 0s 49ms/step - loss: 1.1440
Epoch 7/100
1/1 [==============================] - 0s 47ms/step - loss: 1.1413
Epoch 8/100
1/1 [==============================] - 0s 45ms/step - loss: 1.1386
Epoch 9/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1360
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1333
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1306
Epoch 12/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1280
Epoch 13/100
1/1 [==============================] - 0s 24ms/

2024-02-11 15:59:26.641345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
